# Update coregistration constants

For tiling purposes, we use a default set of constants to coregister FOV coordinates onto the slide. However, depending on the positioning of your slide on the commercial instrument, you may need to adjust these.

Coregistration is done using the fiducial markers on the slide. Please provide the stage coordinate and corresponding optical pixel coordinate values for the top 3 pairs in this notebook.

To retrieve the stage coordinate and pixel coordinate values, open the slide on the CAC and open the developer console. Set `logger.level = 4`, then click on one of the fiducials. The stage coordinates will be printed above the slide, and the corresponding pixel coordinates will be printed in the console.

In [1]:
import numpy as np
import os
from sklearn.linear_model import LinearRegression

### Set stage and pixel coordinates for the top 3 fiducial pairs

TODO: prompt the user instead?

In [2]:
# top pair
top_pair_left_stage = (0.1, 49)
top_pair_left_pixel = (405, 410)
top_pair_right_stage = (22, 48)
top_pair_right_pixel = (723, 414)

# middle pair
middle_pair_left_stage = (0.2, 33)
middle_pair_left_pixel = (401, 641)
middle_pair_right_stage = (22, 32)
middle_pair_right_pixel = (721, 644)

# bottom pair
bottom_pair_left_stage = (0.3, 18)
bottom_pair_left_pixel = (403, 875)
bottom_pair_right_stage = (22, 17)
bottom_pair_right_pixel = (722, 878)

### Regress stage to pixel coordinates (separately for x- and y-coordinate values)

In [3]:
# define the lists for x-coordinate stage and corresponding pixel values for regression
x_stage = np.array([
    top_pair_left_stage[0], top_pair_right_stage[0],
    middle_pair_left_stage[0], middle_pair_right_stage[0],
    bottom_pair_left_stage[0], bottom_pair_right_stage[0]
])
x_pixel = np.array([
    top_pair_left_pixel[0], top_pair_right_pixel[0],
    middle_pair_left_pixel[0], middle_pair_right_pixel[0],
    bottom_pair_left_pixel[0], bottom_pair_right_pixel[0]
])

In [4]:
# define the lists for y-coordinate stage and corresponding pixel values for regression
y_stage = np.array([
    top_pair_left_stage[1], top_pair_right_stage[1],
    middle_pair_left_stage[1], middle_pair_right_stage[1],
    bottom_pair_left_stage[1], bottom_pair_right_stage[1]
])
y_pixel = np.array([
    top_pair_left_pixel[1], top_pair_right_pixel[1],
    middle_pair_left_pixel[1], middle_pair_right_pixel[1],
    bottom_pair_left_pixel[1], bottom_pair_right_pixel[1]
])

In [5]:
# generate regression parameters for x stage to pixel
x_regression = LinearRegression().fit(
    np.reshape(x_stage, (-1, 1)),
    np.reshape(x_pixel, (-1, 1))
)

x_multiplier = x_regression.coef_[0][0]
x_offset = x_regression.intercept_ / x_multiplier

In [6]:
# generate regression parameters for y stage to pixel
y_regression = LinearRegression().fit(
    np.reshape(y_stage, (-1, 1)),
    np.reshape(y_pixel, (-1, 1))
)

y_multiplier = y_regression.coef_[0][0]
y_offset = y_regression.intercept_ / y_multiplier

### Change global fiducial settings

In [7]:
# append new constant values to settings.py
with open(os.path.join('../toffy/settings.py'), 'a') as fw:
    fw.write('STAGE_TO_PIXEL_X_MULTIPLIER = %f\n' % x_multiplier)
    fw.write('STAGE_TO_PIXEL_X_OFFSET = %f\n' % x_offset)
    fw.write('STAGE_TO_PIXEL_Y_MULTIPLIER = %f\n' % y_multiplier)
    fw.write('STAGE_TO_PIXEL_Y_OFFSET = %f\n' % y_offset)